# World Surf League Scores Web Scraping

## Imports

In [1]:
import pandas as pd
import numpy as np
from requests import get
import json
from bs4 import BeautifulSoup
import time



## World Surf League Scores

I'm going to focus on the Pipeline Masters first. It is the marquee event of the Championship Tour and is run every year (**verify**).

### Get HTML soup and review

Use Beautiful Soup to get html from World Surf League event pages. Each page has the final result and a 'carousel' that links to previous rounds.

In [2]:
# Starting URLs
pipe_2008_url = "https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters"
res = get(pipe_2008_url)
soup = BeautifulSoup(res.content)


From inspecting page source I have found an unordered list with useful urls to get all scores from each event

In [3]:
# Find all list items within the event rounds div
carousel = soup.find('div', id='event-rounds').find_all('li')

In [4]:
# Visual check of event round urls and round names/numbers
for horse in carousel:
    link_elements = horse.find_all('a')
    print(link_elements)

[<a class="next" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=4">Round 2</a>]
[<a class="previous" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=3">Round 1</a>, <a class="next" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=5">Round 3</a>]
[<a class="previous" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=4">Round 2</a>, <a class="next" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=6">Round 4</a>]
[<a class="previous" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pipeline-masters?roundId=5">Round 3</a>, <a class="next" data-no-scroll="true" data-request-name="eventRound" href="/events/2008/mct/75/billabong-pip

In [5]:
# Build a dict of urls and round names
round_urls = []
for horse in carousel:
    link_elements = horse.find_all('a')
    for link in link_elements:
        url_dict = {}
        url_dict['round'] = link.text
        # Add endpoint for ease of use later
        url_dict['url'] = "https://www.worldsurfleague.com" + link['href']
        print(link['href'], link.text)
        round_urls.append(url_dict)

/events/2008/mct/75/billabong-pipeline-masters?roundId=4 Round 2
/events/2008/mct/75/billabong-pipeline-masters?roundId=3 Round 1
/events/2008/mct/75/billabong-pipeline-masters?roundId=5 Round 3
/events/2008/mct/75/billabong-pipeline-masters?roundId=4 Round 2
/events/2008/mct/75/billabong-pipeline-masters?roundId=6 Round 4
/events/2008/mct/75/billabong-pipeline-masters?roundId=5 Round 3
/events/2008/mct/75/billabong-pipeline-masters?roundId=7 Quarterfinal
/events/2008/mct/75/billabong-pipeline-masters?roundId=6 Round 4
/events/2008/mct/75/billabong-pipeline-masters?roundId=52 Semifinal
/events/2008/mct/75/billabong-pipeline-masters?roundId=7 Quarterfinal
/events/2008/mct/75/billabong-pipeline-masters?roundId=77 Final
/events/2008/mct/75/billabong-pipeline-masters?roundId=52 Semifinal


In [6]:
# With help from https://stackoverflow.com/questions/11092511/list-of-unique-dictionaries
# Keep only unique values
round_urls_unique = list({unique['url']:unique for unique in round_urls}.values())
round_urls_unique

[{'round': 'Round 2',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=4'},
 {'round': 'Round 1',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=3'},
 {'round': 'Round 3',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=5'},
 {'round': 'Round 4',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=6'},
 {'round': 'Quarterfinal',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=7'},
 {'round': 'Semifinal',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=52'},
 {'round': 'Final',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters?roundId=77'}]

### Get scores for every round in an event

In [7]:
# Get scores function takes the url dict and gets scores for each round of the event
def get_scores(url_list_of_dicts, event_year, event_name):
    data = []
    # Iterate through event rounds
    for round in url_list_of_dicts:
        # Get html soup
        res = get(round["url"])
        round_soup = BeautifulSoup(res.content)
        # Get list of athlete names
        athletes = round_soup.find_all("span", class_="athlete-name")

        # Get list of scores lists
        scores = round_soup.find_all("div", class_="all-waves all-waves-grid")

        # Go through athletes
        for i in range(len(athletes)):

            # Create dict to save scores
            athlete_scores = {}

            # Save event name and year
            athlete_scores["year"] = event_year
            athlete_scores["event"] = event_name

            # Save round name
            athlete_scores["round"] = round["round"]

            # Save heat number
            athlete_scores["heat"] = (
                athletes[i]
                .find_parent("div", class_="bd new-heat-bd")
                .find_previous_sibling()
                .find("span", class_="new-heat-hd-name")
                .text
            )

            # Athlete name
            athlete_scores["name"] = athletes[i].text

            # Empty list for athletes scores in that heat
            score_list = []

            # Scores list
            per_wave_scores = scores[i].find_all("span", class_="score")

            # Append scores
            for b in range(len(per_wave_scores)):
                try:
                    score_list.append(float(per_wave_scores[b].text))
                except:
                    continue

            # Get two top scores of each athlete per heat
            if len(score_list) > 0:
                top_score = max(score_list)
                score_list.remove(top_score)
                if len(score_list) > 0:
                    second_score = max(score_list)
                    # Assign total score to data
                    athlete_scores["top_two_waves_total"] = top_score + second_score
                else:
                    athlete_scores["top_two_waves_total"] = top_score
            else:
                # Assign total score to data
                athlete_scores["top_two_waves_total"] = 0.0

            # Add to data list
            data.append(athlete_scores)

        # Sleep
        time.sleep(1)

    return data


In [8]:
pipe_2008_scores = get_scores(round_urls_unique, '2008', 'pipe')

In [9]:
pd.DataFrame(pipe_2008_scores)

,year,event,round,heat,name,top_two_waves_total
0,2008,pipe,Round 2,Heat 1,Kamalei Alexander,11.90
1,2008,pipe,Round 2,Heat 1,Mikael Picon,11.67
2,2008,pipe,Round 2,Heat 2,Evan Valiere,14.57
3,2008,pipe,Round 2,Heat 2,Mick Campbell,14.34
4,2008,pipe,Round 2,Heat 3,Jamie O'Brien,15.00
...,...,...,...,...,...,...
122,2008,pipe,Semifinal,Heat 1,Tim Reyes,15.60
123,2008,pipe,Semifinal,Heat 2,Chris Ward,16.46
124,2008,pipe,Semifinal,Heat 2,Adrian Buchan,10.16
125,2008,pipe,Final,Heat 1,Kelly Slater,14.00


### Get all scores for all Pipeline events in all years

In [10]:
# Get urls for each year's events page
yearly_urls = []
for year in range(2008, 2023):
    # Exclude 2020, all events were cancelled
    if year != 2020:
        dict_list = {}
        dict_list['year'] = year
        dict_list['url'] = f'https://www.worldsurfleague.com/events/{year}/mct?all=1'
        yearly_urls.append(dict_list)

In [11]:
# Get URL for each event accross years
event_urls = []
for year in yearly_urls:
    res = get(year['url'])
    soup = BeautifulSoup(res.content)
    for each in soup.find_all("a", class_="event-schedule-details__event-name"):
        # Only get pipeline events
        if 'Pipe' in each.text.replace('/',''):
            event = {}
            event['year'] = year['year']
            event['name'] = each.text.replace('/','')
            event['url'] = each['href']
            event_urls.append(event)
        else:
            continue
    time.sleep(1)

event_urls

[{'year': 2008,
  'name': 'Billabong Pipeline Masters',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/75/billabong-pipeline-masters'},
 {'year': 2009,
  'name': 'Billabong Pipeline Masters',
  'url': 'https://www.worldsurfleague.com/events/2009/mct/138/billabong-pipeline-masters'},
 {'year': 2010,
  'name': 'Billabong Pipe Masters',
  'url': 'https://www.worldsurfleague.com/events/2010/mct/220/billabong-pipe-masters'},
 {'year': 2011,
  'name': 'Billabong Pipe Masters',
  'url': 'https://www.worldsurfleague.com/events/2011/mct/368/billabong-pipe-masters'},
 {'year': 2012,
  'name': 'Billabong Pipe Masters',
  'url': 'https://www.worldsurfleague.com/events/2012/mct/504/billabong-pipe-masters'},
 {'year': 2013,
  'name': 'Billabong Pipe Masters',
  'url': 'https://www.worldsurfleague.com/events/2013/mct/615/billabong-pipe-masters'},
 {'year': 2014,
  'name': 'Billabong Pipe Masters',
  'url': 'https://www.worldsurfleague.com/events/2014/mct/730/billabong-pipe-masters'},
 {'ye

In [12]:
# Get all scores and save them as a list of df
df_list = []
for event in event_urls:
    print(f"Getting {event['year']} {event['name']}...")
    res = get(event["url"])
    soup = BeautifulSoup(res.content)
    # Find all list items within the event rounds div
    try:
        carousel = soup.find("div", id="event-rounds").find_all("li")
    except:
        print(f"FAILED {event['year']} {event['name']}")
        continue

    # Build a dict of urls and round names
    round_urls = []
    for horse in carousel:
        link_elements = horse.find_all("a")
        for link in link_elements:
            url_dict = {}
            url_dict["round"] = link.text
            # Add endpoint for ease of use later
            url_dict["url"] = "https://www.worldsurfleague.com" + link["href"]
            round_urls.append(url_dict)

    # Keep only unique values
    round_urls_unique = list({unique["url"]: unique for unique in round_urls}.values())

    # Run get scores function and assign result to a variable
    event_scores = get_scores(
        round_urls_unique, event_year=event["year"], event_name=event["name"]
    )

    # Convert scores data to DataFrame and append to df_list
    # pd.DataFrame(event_scores).to_csv(
    #     f"./data/{event['name']} {event['year']}.csv", index=False
    # )
    df_list.append(pd.DataFrame(event_scores))
    print(f"... got {event['year']} {event['name']}")


Getting 2008 Billabong Pipeline Masters...
... got 2008 Billabong Pipeline Masters
Getting 2009 Billabong Pipeline Masters...
... got 2009 Billabong Pipeline Masters
Getting 2010 Billabong Pipe Masters...
... got 2010 Billabong Pipe Masters
Getting 2011 Billabong Pipe Masters...
... got 2011 Billabong Pipe Masters
Getting 2012 Billabong Pipe Masters...
... got 2012 Billabong Pipe Masters
Getting 2013 Billabong Pipe Masters...
... got 2013 Billabong Pipe Masters
Getting 2014 Billabong Pipe Masters...
... got 2014 Billabong Pipe Masters
Getting 2015 Billabong Pipe Masters...
... got 2015 Billabong Pipe Masters
Getting 2016 Billabong Pipe Masters...
... got 2016 Billabong Pipe Masters
Getting 2017 Billabong Pipe Masters...
... got 2017 Billabong Pipe Masters
Getting 2018 Billabong Pipe Masters...
... got 2018 Billabong Pipe Masters
Getting 2019 Billabong Pipe Masters...
... got 2019 Billabong Pipe Masters
Getting 2021 Billabong Pipe Masters presented by Hydro Flask...
... got 2021 Billabo

In [13]:
pd.concat(df_list)

,year,event,round,heat,name,top_two_waves_total
0,2008,Billabong Pipeline Masters,Round 2,Heat 1,Kamalei Alexander,11.90
1,2008,Billabong Pipeline Masters,Round 2,Heat 1,Mikael Picon,11.67
2,2008,Billabong Pipeline Masters,Round 2,Heat 2,Evan Valiere,14.57
3,2008,Billabong Pipeline Masters,Round 2,Heat 2,Mick Campbell,14.34
4,2008,Billabong Pipeline Masters,Round 2,Heat 3,Jamie O'Brien,15.00
...,...,...,...,...,...,...
105,2022,Billabong Pro Pipeline,Semifinals,Heat 1 Watch replay,Miguel Pupo,9.50
106,2022,Billabong Pro Pipeline,Semifinals,Heat 2 Watch replay,Seth Moniz,13.50
107,2022,Billabong Pro Pipeline,Semifinals,Heat 2 Watch replay,Caio Ibelli,6.33
108,2022,Billabong Pro Pipeline,Final,Heat 1 Watch replay,Kelly Slater,18.77


In [14]:
pd.concat(df_list).to_csv('../data/wsl/pipeline_avg_scores_all_years.csv')

## World Surf League Dates and Times of Rounds

Using a similar loop as before, get dates for event heats where possible

In [15]:
res = get(event_urls[-4]["url"] + "/results")
soup = BeautifulSoup(res.content)
results_grid = soup.find(
    "div", class_="post-event-watch-heat-grid__body post-event-watch-module-body"
)
round_name = results_grid.find(
    "div", class_="post-event-watch-heat-grid__round-header"
).text.strip()
heats = results_grid.find_all("div", class_="hot-heat__hd")


In [16]:
heats[2].find("span", class_="heat-name").text


'Heat 3'

In [17]:
heats[2].find("span", class_="hot-heat__status hot-heat__status--over").text


'Ended December 12, 2018'

In [18]:
# Find carousel and get round urls for each event
carousel = soup.find("div", class_="post-event-watch-round-nav post-event-watch-module")
links = carousel.find_all("a")
for link in links:
    print(link['href'])


/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12329
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12330
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12331
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12332
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12333
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12334
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12335
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12429
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12430
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12431
/events/2018/mct/2856/billabong-pipe-masters/results?roundId=12432


In [19]:
# Build a data frame with year,event,round,heat,date columns for all events all years
df_list = []
for event in event_urls:
    data = []
    print(f"Getting {event['year']} {event['name']}...")
    res = get(event["url"] + "/results")
    soup = BeautifulSoup(res.content)
    carousel = soup.find(
        "div", class_="post-event-watch-round-nav post-event-watch-module"
    )
    links = carousel.find_all("a")
    for link in links:
        endpoint = link["href"].split("?")[1]
        res2 = get(event["url"] + "/results?" + endpoint)
        soup2 = BeautifulSoup(res2.content)
        results_grid = soup2.find(
            "div",
            class_="post-event-watch-heat-grid__body post-event-watch-module-body",
        )

        if results_grid:
            round_name = results_grid.find(
                "div", class_="post-event-watch-heat-grid__round-header"
            ).text.strip()
            heats = results_grid.find_all("div", class_="hot-heat__hd")
            for heat in heats:
                heat_data = {}
                try:
                    heat_data["year"] = event["year"]
                    heat_data["event"] = event["name"]
                    heat_data["round"] = round_name
                    try:
                        heat_data["heat"] = heat.find("span", class_="heat-name").text
                    except:
                        heat_data["heat"] = 'Heat 1'
                    heat_data["date"] = heat.find(
                        "span", class_="hot-heat__status hot-heat__status--over"
                    ).text.replace('Ended', '')
                    data.append(heat_data)
                    print(heat_data['heat'], heat_data['date'])
                except:
                    print(
                        f"something wrong in heat data for {event['year']} {event['name']}"
                    )
                    continue
        else:
            print(f"FAILED {event['year']} {event['name']}")
            continue
        time.sleep(1)
    print(f"... got {event['year']} {event['name']}")

    df_list.append(pd.DataFrame(data))
    time.sleep(2)


Getting 2008 Billabong Pipeline Masters...
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 9  
Heat 10  
Heat 11  
Heat 12  
Heat 13  
Heat 14  
Heat 15  
Heat 16  
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 9  
Heat 10  
Heat 11  
Heat 12  
Heat 13  
Heat 14  
Heat 15  
Heat 16  
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 9  
Heat 10  
Heat 11  
Heat 12  
Heat 13  
Heat 14  
Heat 15  
Heat 16  
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 1  
Heat 2  
Heat 1  
... got 2008 Billabong Pipeline Masters
Getting 2009 Billabong Pipeline Masters...
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 9  
Heat 10  
Heat 11  
Heat 12  
Heat 13  
Heat 14  
Heat 15  
Heat 16  
Heat 1  
Heat 2  
Heat 3  
Heat 4  
Heat 5  
Heat 6  
Heat 7  
Heat 8  
Heat 9  
Heat 10  
Heat 11  
Heat 12  
Heat 13  
Heat 14  
Heat

In [20]:
dates = pd.concat(df_list)
dates[dates['year'] == 2022]['date'].value_counts()

 February 1, 2022    24
 January 29, 2022    16
 February 5, 2022     7
Name: date, dtype: int64

In [21]:
dates[dates['year'] == 2022]['round'].value_counts()

Round of 32          16
Opening Round        12
Round of 16           8
Elimination Round     4
Quarterfinals         4
Semifinals            2
Final                 1
Name: round, dtype: int64

In [22]:
dates['round'].value_counts()

Round 1                 140
Round 3                 140
Round 2                 140
Round 4                  52
Round of 32              48
Quarterfinal             36
Round 5                  32
Seeding Round            24
Round of 16              24
Quarterfinals            20
Semifinal                18
Final                    14
Elimination Round        12
Opening Round            12
Semifinals               10
Trials Round 1            8
Trials Quarterfinals      4
Trials Semifinals         2
Trials Final              1
Name: round, dtype: int64

In [23]:
dates['heat'].value_counts()

Heat 1     110
Heat 2      95
Heat 3      80
Heat 4      80
Heat 5      45
Heat 6      45
Heat 7      45
Heat 8      45
Heat 9      39
Heat 10     39
Heat 11     39
Heat 12     39
Heat 13      9
Heat 14      9
Heat 15      9
Heat 16      9
Name: heat, dtype: int64

In [24]:
dates.to_csv('../data/wsl/pipe_masters_heat_dates.csv')

## All events all years

### Get all scores for all events in all years

In [25]:
# Get urls for each year's events page
yearly_urls = []
for year in range(2008, 2023):
    # Exclude 2020, all events were cancelled
    if year != 2020:
        dict_list = {}
        dict_list['year'] = year
        dict_list['url'] = f'https://www.worldsurfleague.com/events/{year}/mct?all=1'
        yearly_urls.append(dict_list)

In [26]:
# Get URL for each event accross years
event_urls = []
for year in yearly_urls:
    res = get(year['url'])
    soup = BeautifulSoup(res.content)
    for each in soup.find_all("a", class_="event-schedule-details__event-name"):
        event = {}
        event['year'] = year['year']
        event['name'] = each.text.replace('/','')
        event['url'] = each['href']
        event_urls.append(event)
        
    time.sleep(1)

event_urls

[{'year': 2008,
  'name': 'Quiksilver Pro Gold Coast',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/4/quiksilver-pro-gold-coast'},
 {'year': 2008,
  'name': 'Rip Curl Pro Bells Beach',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/8/rip-curl-pro-bells-beach'},
 {'year': 2008,
  'name': 'Billabong Pro Teahupoo',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/20/billabong-pro-teahupoo'},
 {'year': 2008,
  'name': 'Globe Pro Fiji',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/24/globe-pro-fiji'},
 {'year': 2008,
  'name': 'Billabong Pro J-Bay',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/31/billabong-pro-j-bay'},
 {'year': 2008,
  'name': 'Rip Curl Pro Search Bali',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/37/rip-curl-pro-search-bali'},
 {'year': 2008,
  'name': 'Boost Mobile Pro',
  'url': 'https://www.worldsurfleague.com/events/2008/mct/48/boost-mobile-pro'},
 {'year': 2008,
  'name': 'Quiksilver Pro France'

In [27]:
# Get all scores and save them as a list of df
df_list = []
for event in event_urls:
    print(f"Getting {event['year']} {event['name']}...")
    res = get(event["url"])
    soup = BeautifulSoup(res.content)
    # Find all list items within the event rounds div
    try:
        carousel = soup.find("div", id="event-rounds").find_all("li")
    except:
        print(f"FAILED {event['year']} {event['name']}")
        continue

    # Build a dict of urls and round names
    round_urls = []
    for horse in carousel:
        link_elements = horse.find_all("a")
        for link in link_elements:
            url_dict = {}
            url_dict["round"] = link.text
            # Add endpoint for ease of use later
            url_dict["url"] = "https://www.worldsurfleague.com" + link["href"]
            round_urls.append(url_dict)

    # Keep only unique values
    round_urls_unique = list({unique["url"]: unique for unique in round_urls}.values())

    # Run get scores function and assign result to a variable
    event_scores = get_scores(
        round_urls_unique, event_year=event["year"], event_name=event["name"]
    )

    
    # pd.DataFrame(event_scores).to_csv(
    #     f"./data/{event['name']} {event['year']}.csv", index=False
    # )

    # Convert scores data to DataFrame and append to df_list
    df_list.append(pd.DataFrame(event_scores))
    print(f"... got {event['year']} {event['name']}")


Getting 2008 Quiksilver Pro Gold Coast...
... got 2008 Quiksilver Pro Gold Coast
Getting 2008 Rip Curl Pro Bells Beach...
... got 2008 Rip Curl Pro Bells Beach
Getting 2008 Billabong Pro Teahupoo...
... got 2008 Billabong Pro Teahupoo
Getting 2008 Globe Pro Fiji...
... got 2008 Globe Pro Fiji
Getting 2008 Billabong Pro J-Bay...
... got 2008 Billabong Pro J-Bay
Getting 2008 Rip Curl Pro Search Bali...
... got 2008 Rip Curl Pro Search Bali
Getting 2008 Boost Mobile Pro...
... got 2008 Boost Mobile Pro
Getting 2008 Quiksilver Pro France...
... got 2008 Quiksilver Pro France
Getting 2008 Billabong Pro Mundaka...
... got 2008 Billabong Pro Mundaka
Getting 2008 Hang Loose Santa Catarina Pro...
... got 2008 Hang Loose Santa Catarina Pro
Getting 2008 Billabong Pipeline Masters...
... got 2008 Billabong Pipeline Masters
Getting 2009 Quiksilver Pro Gold Coast...
... got 2009 Quiksilver Pro Gold Coast
Getting 2009 Rip Curl Pro Bells Beach...
... got 2009 Rip Curl Pro Bells Beach
Getting 2009 Bill

In [28]:
pd.concat(df_list)

,year,event,round,heat,name,top_two_waves_total
0,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Andy Irons,16.37
1,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Tamaroa McComb,5.90
2,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Julian Wilson,14.00
3,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Pancho Sullivan,13.23
4,2008,Quiksilver Pro Gold Coast,Round 2,Heat 3,Jeremy Flores,10.83
...,...,...,...,...,...,...
105,2022,MEO Pro Portugal,Semifinals,Heat 1 Watch replay,Italo Ferreira,12.66
106,2022,MEO Pro Portugal,Semifinals,Heat 2 Watch replay,Griffin Colapinto,13.76
107,2022,MEO Pro Portugal,Semifinals,Heat 2 Watch replay,John John Florence,10.50
108,2022,MEO Pro Portugal,Final,Heat 1 Watch replay,Griffin Colapinto,14.34


In [29]:
pd.concat(df_list).to_csv('../data/wsl/avg_scores_all_years.csv')